# Build a Multi-Agent Systems
In this notebook, we'll:
- ✅ Configure the API key to use the Gemini model
- ✅ Learn when to use multi-agent systems in [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Build a system using an LLM as a "manager"
- ✅ Learn three core workflow patterns (Sequential, Parallel, and Loop) to coordinate the agent teams

## Part 1: Add Gemini API key to the env

This notebook uses the Gemini API, which requires authentication.

1. Get your API key

If you don't have one already, create an API key in Google AI Studio.

2. Add the key to env and load as shown in the script 

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

google_api_key = os.environ.get("GOOGLE_API_KEY")
if not google_api_key:
    print(
        "GOOGLE_API_KEY is not set. Create one in Google AI Studio "
        "(https://aistudio.google.com/) and add it to your env or .env file."
    )
    raise SystemExit("Set GOOGLE_API_KEY and rerun this cell.")

print("✅ GOOGLE_API_KEY loaded.")

✅ GOOGLE_API_KEY loaded.


### validate the api key

In [2]:
from google import genai
from google.genai import types

client = genai.Client(api_key=google_api_key)

try:
    _ = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents="ping",
        config=types.GenerateContentConfig(max_output_tokens=1),
    )
    print("✅ API key validated with Gemini.")
except Exception as err:
    raise SystemExit(f"API key validation failed: {err}")

✅ API key validated with Gemini.


### Import ADK components

In [3]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## Part 2: Why multi-agent systems? + Design a multi-agent
**The Problem: The "Do-It-All" Agent**
While single agents can accomplish many tasks, they quickly reach limitations as tasks grow in complexity. A monolithic agent that attempts to handle research, writing, editing, and fact-checking simultaneously faces several challenges:
- Bloated and confusing instruction prompts
- Difficulty in debugging (unclear which part failed)
- Maintenance challenges
- Higher risk of producing unreliable results

**The Solution: A Team of Specialists**
Rather than relying on a single agent for every task, a **multi-agent system** employs several specialized agents that work together—just like a collaborative team. Each agent has one clear responsibility (for example, one agent focuses on research, another handles writing). This specialization leads to:
- Easier construction and testing of individual agents
- Greater reliability and maintainability
- Enhanced overall system performance through collaboration

For further details, refer to the [LLM agents in ADK documentation](https://google.github.io/adk-docs/agents/llm-agents/).
**Architecture: Single Agent vs Multi-Agent Team**


### Example: Research + Summarization System
Let's build a system with two specialized agents:

1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings

### Other Multi-Agent System Examples
<!-- #
# Here are several ideas for multi-agent systems composed of specialized agents:
#
# 1. **Question Answering System**
#    - **Retriever Agent**: Finds relevant documents or passages from a knowledge base.
#    - **Reader Agent**: Extracts and formulates a direct answer from the retrieved text.
#
# 2. **Content Generation Pipeline**
#    - **Research Agent**: Gathers key facts and background information.
#    - **Writer Agent**: Generates a draft article or story based on research.
#    - **Editor Agent**: Refines grammar, style, and coherence.
#
# 3. **Multi-Modal Assistant**
#    - **Image Analysis Agent**: Interprets and describes uploaded images.
#    - **Text Summarization Agent**: Summarizes content from images and user input.
#    - **Fact-Checking Agent**: Verifies the accuracy of generated statements.
#
# 4. **Customer Support System**
#    - **Intent Recognition Agent**: Classifies user requests.
#    - **FAQ Lookup Agent**: Searches a database for relevant answers.
#    - **Escalation Agent**: Determines when to hand off to a human or higher-level agent.
#
# 5. **Creative Design Team**
#    - **Idea Generator Agent**: Proposes new concepts or approaches.
#    - **Critique Agent**: Reviews and suggests improvements. -->

In [4]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings", # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")


✅ research_agent created.


In [5]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")


✅ summarizer_agent created.


Then we bring the agents together under a root agent, or coordinator:

In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")

✅ root_agent created.
